### Setup and Imports

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

import config
import pandas as pd
from src import data_ingestion
from src.symbols import Symbols
from src.portfolio import Portfolio
from src.benchmark import Benchmark

### Load Transaction Data

In [ ]:
# Create a list of all your transaction log files
log_files = [
    config.TRANS_LOG_DIR / "us_mkt_transactions.csv",
    config.TRANS_LOG_DIR / "exus_mkt_transactions.csv"
]

# Load and merge all found logs into a single DataFrame
master_log = data_ingestion.create_master_log(log_files).set_index("Date")
display(master_log)

Successfully loaded log: us_mkt_transactions.csv
Successfully loaded log: exus_mkt_transactions.csv


,Type,Symbol,Quantity,Price,Amount,Commission,Currency,Description,Exchange,Source
Date,,,,,,,,,,
2023-02-16,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,ID: 37fdafdc-d707-42fa-ba74-98b30cf9ab2a - DT2...,US Market,Sarwa Trade
2023-02-17,buy,PERI,0.407270,33.340000,-13.58,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-02-17,buy,TSLA,0.202666,201.020000,-40.74,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-02-17,buy,VOO,0.218967,372.070000,-81.47,NaN,USD,Trade Entry,NYSEArca,Sarwa Trade
2023-03-29,Net Dividend,VOO,NaN,NaN,0.24,NaN,USD,"Cash DIV @ 1.4874, Pos QTY: 0.218966527, Rec D...",NYSEArca,Sarwa Trade
...,...,...,...,...,...,...,...,...,...,...
2025-07-24,buy,BRK-B,0.124000,481.935484,-59.76,1.0,USD,Purchase of BRK-B,NYSE,Sarwa Trade - Interim
2025-07-24,buy,BTC-USD,0.000162,120615.384615,-19.60,NaN,USD,NaN,CCC,Sarwa Crypto
2025-07-24,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,Bank Deposit,US Market,Sarwa Trade - Interim


### Initialize and Assess Symbols

In [3]:
# 1. Initialize the Symbol manager
symbol_manager = Symbols(master_log)

# 2. Assess all symbols (checks cache and yfinance, creates templates for missing symbols)
symbol_manager.assess()

# 3. Display the results for user verification
print("✅ The following symbols were automatically identified:")
found_df = symbol_manager.get_found()
display(found_df)

print("❌ The following symbols were NOT found:")
missing_list = symbol_manager.get_missing()
print(missing_list)

✅ The following symbols were automatically identified:


,Name,Type,Exchange,Currency,Industry,Sector,Country,DataProvider
Symbol,,,,,,,,
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,Israel,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,United States,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance
...,...,...,...,...,...,...,...,...
JPMB,JPMorgan USD Emerging Markets Sovereign Bond ETF,etf,NYSEArca,USD,None,None,None,yfinance
SMOT,VanEck Morningstar SMID Moat ETF,etf,Cboe US,USD,None,None,None,yfinance
XLV,The Health Care Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance


❌ The following symbols were NOT found:
['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY']


### User Correction Step

Reviews the output from the cell above and lists any symbols that were incorrectly identified by the data provider (e.g. `yfinance`).

In [4]:
# User reviews the table above and populates this list
incorrectly_identified_symbols = [
    'AMR', 'ASM'
]

# Mark these symbols as user-provided, which also updates the metadata template
symbol_manager.mark_as_manual(incorrectly_identified_symbols)

Updating cache for incorrectly identified symbols: ['AMR', 'ASM']
Caches updated successfully.


### Manual Task

1) Open the file: `data/manual-source/metadata.json`.
2) Fill in the `null` values for all the symbols listed.
3) For each of those same symbols, add a price history CSV file to the `data/manual-source/` directory.

### Reload Metadata and Get Final DataFrame

In [5]:
# Reload the user-edited file from disk
symbol_manager.reload_user_metadata()

# Get the final, combined DataFrame of all symbol metadata
symbol_df = symbol_manager.get_unified_df()

print("--- Final Unified Symbol Metadata ---")
display(symbol_df)

Reloading user-provided metadata from disk...
User metadata reloaded successfully.
Successfully created unified symbols DataFrame.
--- Final Unified Symbol Metadata ---


,Name,Type,Exchange,Currency,Industry,Sector,Country,DataProvider
Symbol,,,,,,,,
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,Israel,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,United States,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance
...,...,...,...,...,...,...,...,...
IHC,International Holding Company PJSC,equity,ADX,AED,Industrials,Conglomerates,United Arab Emirates,manual
EMAAR,Emaar Properties PJSC,equity,DFM,AED,Real Estate - Development,Real Estate,United Arab Emirates,manual
MULTIPLY,Multiply Group PJSC,equity,ADX,AED,Financials,Asset Management,United Arab Emirates,manual


### Initialize and Run Portfolio Analysis

In [6]:
# Get the project date range
start_date, end_date, date_range, last_market_day = config.project_dates(
    master_log.index
)

# 1. Initialize the Portfolio analysis engine
portfolio = Portfolio(master_log, symbol_manager, date_range, last_market_day)

# 2. Run all calculations (fetching prices, calculating holdings and value)
portfolio.calculate_holdings_and_value()

Portfolio object initialized.
Preparing trade log...
Fetching price and split data...
Successfully created unified symbols DataFrame.
Converting prices to base currency (USD)...
Successfully created unified symbols DataFrame.
Fetching FX rate for AEDUSD=X...
Calculating daily holdings and value...
Calculating split-adjusted holdings and market value...
Calculations complete.


### Display Results

In [7]:
portfolio.holdings.keys()

dict_keys(['trade', 'price', 'raw_splits', 'holding', 'value', 'adj holding', 'cumulative splits', 'Total Portfolio Value'])

In [8]:
portfolio.holdings['trade']

,PERI,TSLA,VOO,UAE,XLK,BTC-USD,VOOG,NOBL,SPY,VHT,...,ARM,ROM,RYAAY,APO,KKR,JPMB,SMOT,XLV,BX,BRK-B
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-16,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-17,0.40727,0.202666,0.218967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-18,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-19,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-20,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-27,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-07-28,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-07-29,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
portfolio.holdings['adj holding'].iloc[-1].round(12).sort_index()

ADBE     0.000000
ALDAR    0.000000
AMR      5.000000
AMZN     0.149980
APO      0.534729
           ...   
VOOG     0.722997
VXUS     0.716810
XLF      1.309464
XLK      1.223873
XLV      0.289189
Name: 2025-07-31 00:00:00, Length: 70, dtype: float64

In [9]:
portfolio.get_current_holdings()

,Shares,Market Value (USD)
VOO,1.552063e+00,9.017798e+02
XLK,1.223873e+00,3.215605e+02
BTC-USD,3.242000e-04,3.820087e+01
VOOG,7.229965e-01,2.970431e+02
SPY,1.776761e-01,1.123055e+02
NVDA,2.503971e-01,4.453813e+01
LUX,2.000000e+00,4.684800e+01
META,1.385825e-01,1.071852e+02
MSFT,2.558341e-01,1.364875e+02
IBB,4.702170e-02,6.242600e+00
